In [1]:
import torch
from PIL import Image
import faiss
import numpy as np
import clip
import json
import glob
import os

In [4]:
# Kiểm tra và sử dụng GPU nếu có, nếu không thì sử dụng CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
# Tải mô hình CLIP và preprocess
model, preprocess = clip.load("ViT-B/32", device=device)
index_file = 'faiss_normal_ViT.bin'
json_file = 'image_path.json'

In [7]:
# Tải FAISS index và từ điển ánh xạ đã lưu, nếu tồn tại
if os.path.exists(index_file):
    index = faiss.read_index('faiss_normal_ViT.bin')
    if os.path.exists(json_file):
        with open(json_file, 'r') as f:
            id2img_fps = json.load(f)
        current_num_images = index.ntotal
        print(f"Đã tải {current_num_images} ảnh từ FAISS index và image_path.json.")
    else:
        # Nếu chưa có image_path.json, tạo mới
        id2img_fps = {}
        current_num_images = 0
        print(f"FAISS index đã tồn tại, nhưng không có image_path.json. Tạo mới.")
else:
    # Khởi tạo FAISS index mới nếu chưa tồn tại
    dimension = 512  # Kích thước của embedding từ CLIP (512 cho ViT-B/32)
    index = faiss.IndexFlatL2(dimension)
    id2img_fps = {}
    current_num_images = 0
    print("FAISS index mới đã được tạo.")

FAISS index mới đã được tạo.


In [9]:
# Tìm các ảnh mới mà chưa có trong id2img_fps
existing_image_paths = set(id2img_fps.values())
list_image = glob.glob("images/L02/L02_V002/*.jpg")
new_image_paths = [img_path for img_path in list_image if img_path not in existing_image_paths]

print(f"Số lượng ảnh mới cần thêm: {len(new_image_paths)}")

Số lượng ảnh mới cần thêm: 951


In [10]:
# Tạo embedding cho các ảnh mới
new_features = []
new_id2img_fps = {}

for id, image_path in enumerate(new_image_paths, start=current_num_images):
    id2img_fps[id] = image_path
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image).detach().cpu().numpy()
        new_features.append(image_features)
        new_id2img_fps[id] = image_path

if new_features:
    new_image_embeddings = np.concatenate(new_features)
    index.add(new_image_embeddings)
    print(f"Đã thêm {len(new_features)} ảnh mới vào FAISS index.")

    # Lưu FAISS index đã cập nhật
    faiss.write_index(index, index_file)
    print(f"Đã lưu FAISS index vào {index_file}")

    # Cập nhật và lưu lại từ điển ánh xạ
    with open(json_file, 'w') as f:
        f.write(json.dumps(id2img_fps))
    print(f"Đã lưu từ điển ánh xạ vào {json_file}")
else:
    print("Không có ảnh mới để thêm.")

Đã thêm 951 ảnh mới vào FAISS index.
Đã lưu FAISS index vào faiss_normal_ViT.bin
Đã lưu từ điển ánh xạ vào image_path.json
